In [1]:
import os
import fitz
import re

from ollama import Client
import faiss
import pandas as pd
import numpy as np
import Stemmer
from tqdm import tqdm
import gradio as gr

from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.retrievers import BaseRetriever, QueryFusionRetriever
from llama_index.core.schema import TextNode, NodeWithScore
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.storage import StorageContext
from llama_index.llms.ollama import Ollama

from IPython.display import Markdown, display

# Connect to Ollama Server

In [2]:
client = Client(
  host='http://localhost:11434',
)

# Settings

In [3]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Ingestion

In [4]:
import json
# Path to the dataset folder
DATASET_PATH = r'/home/jeryl4913/lto_rag_reviewer/notebooks/extracted_text.json'

def get_text_and_metadata(input_path):
    """Load text and metadata from a file and perform chunking."""
    with open(input_path, "r", encoding="utf-8") as f:
        extracted_data = json.load(f)

    texts = []
    metadata = []

    for entry in tqdm(extracted_data, desc="Processing entries"):
        text = entry["text"]
        source_metadata = {
            "source": entry["source"],
            "folder": entry["folder"],
            "file_name": entry["file_name"],
            "page": entry["page"],
            "title": entry["title"],
            "url": entry["url"]
        }
        texts.append(text)
        metadata.append(source_metadata)

    return texts, metadata


In [5]:
docs, metadatas = get_text_and_metadata(DATASET_PATH)

Processing entries: 100%|██████████| 7815/7815 [00:00<00:00, 1008320.59it/s]


In [6]:
documents = [Document(text=docs[t], metadata=metadatas[t]) for t in range(len(docs))]
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

# Embedding and Retrieval

## Dense via FAISS

In [7]:
def generate_embeddings(nodes, client, model):
    # Generate embeddings for documents using Ollama
    for doc in tqdm(nodes):
        response = client.embeddings(prompt=doc.text, model=model)
        doc.embedding = response["embedding"]
    return nodes

In [8]:
class FaissIndexer:
    """
    Faiss-based indexer for efficient similarity search using inner-product (cosine) similarity.

    This class handles the creation and management of a FAISS index from node embeddings.
    
    :ivar faiss_index: The FAISS index for storing and querying embeddings.
    :vartype faiss_index: faiss.IndexFlatIP
    :ivar embedding_dim: Dimensionality of the embeddings.
    :vartype embedding_dim: int
    """

    def __init__(self):
        """
        Initialize the FaissIndexer class.

        :ivar faiss_index: The FAISS index, initialized as None.
        :ivar embedding_dim: The dimension of embeddings, initialized as None.
        """
        self.faiss_index = None
        self.embedding_dim = None

    def normalize_embeddings(self, embeddings):
        """
        Normalize embeddings to have unit L2 norm.

        :param embeddings: Array of embeddings to normalize.
        :type embeddings: np.ndarray
        :return: Normalized embeddings.
        :rtype: np.ndarray
        """
        return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    def build_index(self, nodes):
        """
        Build the FAISS index from a list of nodes containing embeddings.

        :param nodes: List of nodes, where each node contains an `embedding` attribute.
        :type nodes: list
        :raises ValueError: If the nodes list is empty or embeddings are inconsistent.
        """
        if not nodes:
            raise ValueError("Nodes list cannot be empty.")
        
        embeddings = np.array([np.array(node.embedding) for node in nodes])
        normalized_embeddings = self.normalize_embeddings(embeddings)

        self.embedding_dim = normalized_embeddings[0].shape[0]
        self.faiss_index = faiss.IndexFlatIP(self.embedding_dim)  # Inner-product similarity
        self.faiss_index.add(normalized_embeddings)

    def get_index(self):
        """
        Get the FAISS index instance.

        :return: The FAISS index used for similarity search.
        :rtype: faiss.IndexFlatIP
        :raises ValueError: If the index has not been built.
        """
        if self.faiss_index is None:
            raise ValueError("Index has not been built yet. Call 'build_index' first.")
        return self.faiss_index

In [9]:
class FAISSVectorStoreRetriever(BaseRetriever):
    def __init__(self, faiss_index, documents):
        """
        Initialize the FAISS retriever.
        :param faiss_index: The FAISS index containing precomputed embeddings.
        :param documents: List of document chunks.
        :param embeddings: Precomputed embeddings corresponding to the document chunks.
        """
        self.faiss_index = faiss_index
        self.documents = documents

    def _retrieve(self, query_embedding, top_k=5):
        """
        Retrieve the top-k nearest neighbors using the FAISS index.
        :param query_embedding: The embedding of the query.
        :param top_k: Number of top results to retrieve.
        """

        norm_query_embedding = np.array([query_embedding])
        norm_query_embedding /= np.linalg.norm(norm_query_embedding, axis=1, keepdims=True)

        distances, indices = self.faiss_index.search(norm_query_embedding, top_k)
        retrieved_docs = [
            NodeWithScore(node=self.documents[idx], score=1 - dist)
            for idx, dist in zip(indices[0], distances[0])
            if idx != -1
        ]
        return retrieved_docs

In [10]:
# embedding
nodes_embed = generate_embeddings(nodes, client, "mxbai-embed-large")

100%|██████████| 12302/12302 [08:21<00:00, 24.52it/s] 


In [11]:
index = FaissIndexer()
index.build_index(nodes_embed)  # Build index from embeddings
faiss_index = index.get_index()  # Retrieve the FAISS index

# Pass the faiss_index during FaissVectorStore initialization
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index_q = VectorStoreIndex.from_documents(
    documents    # Don’t re-embed doc nodes
)
ollama_llm = Ollama(model="llama3.3", request_timeout=300)
query_engine = index_q.as_query_engine(llm=ollama_llm)

faiss_retriever = FAISSVectorStoreRetriever(
    faiss_index=faiss_index,  # Pass the same faiss_index to the retriever
    documents=nodes_embed
)

## Sparse Embedding via BM25

In [12]:
# bm25_retriever = BM25Retriever.from_defaults(
#     nodes=nodes,
#     similarity_top_k=5,
#     stemmer=Stemmer.Stemmer("english"),
#     language="english",
# )

## Hybrid Retrieval via Reciprocal Rank

In [13]:
def hybrid_embedding(results: dict, top_k: int):
    x = QueryFusionRetriever
    ranked_results = QueryFusionRetriever._reciprocal_rerank_fusion(x, results)
    return ranked_results[:top_k]

# Post Retrieval

## Summarization

In [14]:
def summarize_each_chunk(nodes, client, query, model="llama3.3", parent=False):
    if parent:
        chunks = [doc.text for doc in nodes]
    else:
        chunks = [doc.node.text for doc in nodes]
    summaries = []
    
    for i, chunk in enumerate(chunks):
        prompt = f"""
        Summarize the following text in one concise paragraph, focusing on key points relevant to the query: "{query}".
        
        - Emphasize information directly related to the query.
        - Exclude unrelated, redundant, or speculative details.
        - Do NOT introduce new information or answer the query itself. 
        
        Text:
        {chunk}
        
        Summary:
        """
        
        response = client.generate(model=model, prompt=prompt)
        summary = response['response'].strip()
        summaries.append(summary)

    return summaries

# Generation

In [15]:
def generate_response_with_notice(summaries, query, choices, client, model="llama3.3"):
    # Combine summaries into context block
    context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Use the following summarized information to answer the query accurately and concisely. 
    DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
    If the information is not sufficient to fully address the query, respond ONLY with:
    "The available information is insufficient to provide a complete answer to this query."
    \nPlease answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED.

    Summarized Context:
    {context}
    
    Query:
    {query}
    {choices}
    
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

# Querying

## Query Transforms

In [16]:
hyde = HyDEQueryTransform(include_original=True, llm=ollama_llm)

## Query Generation

In [17]:
docstore = {}

# Store documents using full metadata as the key
for doc in documents:
    key = tuple(doc.metadata.items())  # Convert metadata to tuple for hashable key
    docstore[key] = doc

In [18]:
def get_document_by_chunk_metadata(chunk_node):
    # Convert chunk metadata to tuple for matching
    metadata_key = tuple(chunk_node.metadata.items())

    # Retrieve document from docstore
    document = docstore.get(metadata_key)
    return document

In [19]:
def remove_duplicate_documents(doc_list):
    seen_ids = set()
    unique_docs = []

    for doc in doc_list:
        if doc.doc_id not in seen_ids:
            seen_ids.add(doc.doc_id)
            unique_docs.append(doc)

    return unique_docs

In [20]:
def gen_query(query, choices, top_k, client, mode='dense', summary=False, model="llama3.3", chunks_only=True):
    hyde_query_engine = TransformQueryEngine(query_engine, hyde)
    response_hyde = hyde_query_engine.query(query)
    post_processed_response = f"""
    Query: {query}
    Additional Query: {response_hyde.response}
    """
    display(Markdown(f"<b>{post_processed_response}</b>"))
    response = client.embeddings(prompt=post_processed_response, model="mxbai-embed-large")
    query_embedding = response["embedding"]

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.3', query=query,parent=parent_flag)
        context = summaries

    answer = generate_response_with_notice(context, query, choices, client, model=model)

    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references), context

# Evaluation

In [21]:
# Generate prompts dynamically
def generate_choices(row):
    options = []
    for choice in ['A', 'B', 'C', 'D', 'E']:
        # Check for NaN or blank values
        if pd.notna(row[choice]) and row[choice] != '':
            options.append(f"{choice}. {row[choice]}")
    
    # Construct the prompt with few-shot examples
    choices = "\n".join(options)
    
    return choices

In [22]:
from sklearn.model_selection import train_test_split
# Load the Excel file
file_path = '/home/jeryl4913/lto_rag_reviewer/notebooks/eval/LTO_EXAM.csv'
df = pd.read_csv(file_path)

# Display the first few rows of each set
print("Testing Data:")
print(len(df))
display(df.head())

Testing Data:
60


,Question,A,B,C,D,E,Answer,Choices
0,"If the driver is turning left, he must: (U-tur...",have the right of way,do so slowly with caution,yield to approaching cars,NaN,NaN,C,A. have the right of way\nB. do so slowly with...
1,"When a vehicle starts to skid, what should the...",Immediately step on the brakes,Hold firmly on to the wheel while slowing down...,Turn the wheels tp the opposite the direction ...,NaN,NaN,B,A. Immediately step on the brakes\nB. Hold fir...
2,"If you are parking uphill without a curb, turn...",edge of the street,other side of the street,middle of the street,NaN,NaN,A,A. edge of the street\nB. other side of the st...
3,"When parking downhill, you should turn your fr...",toward the curb of the sidewalk,away from the curb,any direction will do,NaN,NaN,A,A. toward the curb of the sidewalk\nB. away fr...
4,Which of the following is the maximum speed li...,60 kph,80 kph,100 kph,NaN,NaN,C,A. 60 kph\nB. 80 kph\nC. 100 kph


In [23]:
import time

df["AI"] = np.nan
ai_answer = []
context_answer = []
inference_times = []
for i in tqdm(range(len(df))):
    start_time = time.time()
    answ = gen_query(df["Question"].iloc[i], df["Choices"].iloc[i], top_k=15, client=client, mode='dense', model="llama3.3")
    ai_answer.append(answ[0])
    context_answer.append(answ[2])
    end_time = time.time()
    inference_times.append(end_time - start_time)

df["AI"] = ai_answer
df["Inference_times"] = inference_times
df["Context"] = context_answer
print("Updated Testing Data with AI Answers:")
display(df.head())

  0%|          | 0/60 [00:00<?, ?it/s]

<b>
    Query: If the driver is turning left, he must: (U-turn / Left Rule)
    Additional Query: Left Rule
    </b>

using FAISS
using chunks only


  2%|▏         | 1/60 [01:05<1:04:18, 65.40s/it]

<b>
    Query: When a vehicle starts to skid, what should the driver do?
    Additional Query: The driver should remain calm and avoid sudden movements. The specific steps to recover from a skid depend on the type of skid. For a front-wheel skid, the driver should lift off the gas, shift to neutral, steer once traction returns, and then resume driving. For a rear-wheel skid, the driver should stay calm, take their foot off the accelerator, steer in the direction they want to go, ease the steering wheel if the wheels start to slide the other way, and brake appropriately.
    </b>

...

  5%|▌         | 3/60 [04:11<1:20:14, 84.47s/it]

<b>
    Query: When parking downhill, you should turn your front wheels:
    Additional Query: into the curb or toward the sidewalk.
    </b>

using FAISS
using chunks only


  5%|▌         | 3/60 [04:52<1:32:39, 97.54s/it]


KeyboardInterrupt: 

In [ ]:
import re


def process_answers(answers):
    formatted_answers = []
    
    for a in answers:
        
        matches = re.findall(r'\[?\s*([A-E](?:\s*,\s*[A-E])*)\s*\]?', str(a)) # Extract answers like [A, C, D] or [A] or [B, D]
        answers = []
        for match in matches:
            answers.extend(re.split(r'\s*,\s*', match))  # Split by comma and remove spaces
        unique_sorted_answers = sorted(set(answers), key=lambda x: ['A', 'B', 'C', 'D', 'E'].index(x))
        if not unique_sorted_answers:
            formatted_answers.append(None)
        else:
            formatted_answers.append(unique_sorted_answers)
    return formatted_answers

df_results = df[["Question", "Answer", "AI", "Inference_times"]].copy()
df_results['Answer'] = df_results['Answer'].apply(lambda x: x.split(', '))
df_results['AI'] = process_answers(df_results["AI"])
df_results['Answer'] = process_answers(df_results["Answer"])



def calculate_scores(df):
    scores = []
    for index, row in df.iterrows():
        correct_answers = set(row['Answer'] if row['Answer'] is not None else [])
        ai_answers = set(row['AI'] if row['AI'] is not None else [])
        if ai_answers == correct_answers:
            score = 1.0
        else:
            score = 0.0
        scores.append(score)
    
    df['Score'] = scores
    accuracy = scores.count(1.0) / len(scores)
    print(f'Final Score: {scores.count(1.0):.2f}/{len(scores):.2f}')
    print(f'Accuracy: {accuracy:.2f}%')
    return df

# Apply the scoring function
scored_df = calculate_scores(df_results)

# Display the dataframe to verify the results
display(scored_df[['Question', 'Answer', 'AI', 'Score', 'Inference_times']])

Final Score: 13.00/60.00
Accuracy: 0.22%


,Question,Answer,AI,Score,Inference_times
0,"If the driver is turning left, he must: (U-tur...",[C],"[A, B, C]",0.0,121.788177
1,"When a vehicle starts to skid, what should the...",[B],"[A, B, C]",0.0,167.223632
2,"If you are parking uphill without a curb, turn...",[A],"[A, B, C]",0.0,126.676048
3,"When parking downhill, you should turn your fr...",[A],[A],1.0,114.733462
4,Which of the following is the maximum speed li...,[C],"[A, B]",0.0,149.391973
5,Never park or stop at the side of the road wit...,[C],"[A, B]",0.0,112.751427
6,While driving with maximum speed and you have ...,[C],"[A, B, C]",0.0,153.455341
7,What will happen when your rear tire blows out?,[B],[A],0.0,178.702681
8,Operating a public utility vehicle equipped wi...,[A],"[A, B, C]",0.0,130.630817
9,Your speed while driving at night should keep on:,[A],"[A, B]",0.0,153.482348


In [ ]:
# Generate prompts dynamically
def generate_prompt(row):
    
    # Construct the prompt with few-shot examples
    prompt = f"\nActual Question: {row['Question']}\n" 
    prompt += "\nPlease answer the question based on the given context."
    
    return prompt

In [ ]:
# Load the Excel file
file_path = '/home/jeryl4913/lto_rag_reviewer/notebooks/eval/LTO_EXAM_QnA.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')
df['Prompt'] = df.apply(generate_prompt, axis=1)
display(df.head())

,Question,Answer,Prompt
0,Traffic Jam can be prevented if you,Keep opposing lanes open,\nActual Question: Traffic Jam can be prevente...
1,When making a right turn you should,Stay on the outermost lane of the road then si...,\nActual Question: When making a right turn yo...
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,\nActual Question: When you intend to turn rig...
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,\nActual Question: At an intersection with a t...
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,\nActual Question: Graft and corruption in the...


In [ ]:
def generate_response_with_notice(summaries, query, client, model="llama3.3"):
    # Combine summaries into context block
    context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Use the following summarized information to answer the query accurately and concisely. 
    DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
    If the information is not sufficient to fully address the query, respond ONLY with:
    "The available information is insufficient to provide a complete answer to this query."

    Summarized Context:
    {context}
    
    Query:
    {query}
    
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

In [ ]:
def gen_query(query, top_k, client, mode='dense', summary=False, model="llama3.3", chunks_only=True):
    hyde_query_engine = TransformQueryEngine(query_engine, hyde)
    response_hyde = hyde_query_engine.query(query)
    post_processed_response = f"""
    Query: {query}
    Additional Query: {response_hyde.response}
    """
    display(Markdown(f"<b>{post_processed_response}</b>"))
    response = client.embeddings(prompt=post_processed_response, model="mxbai-embed-large")
    query_embedding = response["embedding"]

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.3', query=query,parent=parent_flag)
        context = summaries

    answer = generate_response_with_notice(context, query, client, model=model)

    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references), context

In [ ]:
import time
qr_range = (0,60)
df["AI"] = np.nan
ai_answer = []
inference_times = []

for i in tqdm(range(*qr_range)):
    start_time = time.time()
    ai_answer.append(gen_query(df.loc[i,"Prompt"],  top_k=15, client=client, mode='dense', model="llama3.3"))
    end_time = time.time()
    inference_times.append(end_time - start_time)



  0%|          | 0/60 [00:00<?, ?it/s]

<b>
    Query: 
Actual Question: Traffic Jam can be prevented if you

Please answer the question based on the given context.
    Additional Query: drive safely and follow defensive driving techniques, such as maintaining a safe following distance, signaling your intentions, stopping smoothly, and obeying right-of-way rules, which can help reduce the risk of collisions and contribute to a smoother traffic flow.
    </b>

using FAISS
using chunks only


  2%|▏         | 1/60 [02:47<2:45:04, 167.88s/it]

<b>
    Query: 
Actual Question: When making a right turn you should

Please answer the question based on the given context.
    Additional Query: When making a right turn, you should position your vehicle close to the curb or edge of the road, signal early, check for oncoming traffic and pedestrians, yield to others as necessary, maintain a safe speed, avoid cutting corners, and follow traffic signals and signs. Additionally, use your mirrors and check blind spots to ensure no vehicles or cyclists are in your path.
    </b>

...

 98%|█████████▊| 59/60 [1:32:16<02:12, 132.33s/it]

<b>
    Query: 
Actual Question: Children must be held in approved child restraint if there are

Please answer the question based on the given context.
    Additional Query: twelve years or younger, unless they are at least 4’11” (150 cm) in height.
    </b>

using FAISS
using chunks only


100%|██████████| 60/60 [1:34:18<00:00, 94.32s/it] 
/tmp/ipykernel_593765/2685888494.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '["To prevent traffic jams, it's essential to practice defensive driving techniques, such as maintaining a safe distance from other vehicles, avoiding sudden lane changes, and being mindful of your surroundings. Additionally, respecting traffic rules, being courteous to other drivers, and planning your route in advance can help reduce congestion on the roads.\n\nSome specific tips to prevent traffic jams include:\n\n1. Leaving early to account for potential delays\n2. Avoiding peak hours when possible\n3. Using alternative routes or modes of transportation\n4. Carpooling or using ride-sharing services\n5. Being aware of road closures or construction and planning accordingly\n\nBy following these tips and practicing defensive driving, you can help reduce the likelihood of

ValueError: Must have equal len keys and value when setting with an ndarray

In [ ]:
df.loc[qr_range[0]:qr_range[1]-1, "AI"] = [answ[0] for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, "Context"] = [" ".join(map(str, answ[2])) for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, "Inference_Time"] = inference_times 
df_new = df.loc[qr_range[0]:qr_range[1]-1].copy()
df = df_new.copy()

# 10. Similarity Evaluation

In [ ]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator
from llama_index.core.base.embeddings.base import BaseEmbedding
import asyncio
from llama_index.core.embeddings import resolve_embed_model
from pydantic import PrivateAttr

class OllamaEmbeddingModel(BaseEmbedding):
    _client: Client = PrivateAttr()

    def __init__(self, model_name: str = "mxbai-embed-large", timeout: int = 300):
        super().__init__()
        self.model_name = model_name
        self._client = Client() 

    async def _aget_query_embedding(self, query: str) -> list[float]:
        return await self._aget_text_embedding(query)

    async def _aget_text_embedding(self, text: str) -> list[float]:
        loop = asyncio.get_event_loop()
        embedding_response = await loop.run_in_executor(
            None, self._client.embeddings, self.model_name, text
        )
        return embedding_response['embedding']  

    def _get_query_embedding(self, query: str) -> list[float]:
        return self._get_text_embedding(query)

    def _get_text_embedding(self, text: str) -> list[float]:
        embedding_response = self._client.embeddings(
            model=self.model_name,
            prompt=text
        )
        return embedding_response['embedding']


embed_model = OllamaEmbeddingModel(model_name="mxbai-embed-large")
evaluator = SemanticSimilarityEvaluator(
    embed_model=embed_model,
    similarity_threshold=0.6
)

results_scores = []
results_passing = []
for i in tqdm(range(len(df))):
    response = df.loc[i, "AI"]
    reference = df.loc[i, "Answer"]

    result = await evaluator.aevaluate(
    response=response,
    reference=reference,
    )
    results_scores.append(result.score)
    results_passing.append(result.passing)
    
df['Score'] = results_scores
df['Passing'] = results_passing

average_score = df['Score'].mean()
total_items = len(df)
passing_items = df['Passing'].sum()  
print(f"Average Score: {average_score:.4f}")
print(f"Passing: {passing_items}/{total_items}")
display(df[['Question', 'Answer', 'AI', 'Score', 'Passing', 'Inference_Time']])

100%|██████████| 60/60 [00:06<00:00,  9.29it/s]

Average Score: 0.6590
Passing: 48/60


,Question,Answer,AI,Score,Passing,Inference_Time
0,Traffic Jam can be prevented if you,Keep opposing lanes open,"To prevent traffic jams, it's essential to pra...",0.696911,True,167.875098
1,When making a right turn you should,Stay on the outermost lane of the road then si...,"When making a right turn, you should drive in ...",0.779041,True,155.435959
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,"30 meters before turning. \n\nHowever, since t...",0.794384,True,107.802828
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,"At an intersection with a traffic light, make ...",0.776484,True,112.805041
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,To eliminate graft and corruption in the traff...,0.607796,True,171.503720
5,"On a four(4) lane road with single white line,...",Overtake by passing over the solid white line,"On a four-lane road with a single white line, ...",0.827382,True,120.294474
6,A double solid yellow line with broken white l...,Absolutely no overtaking,A double solid yellow line with a broken white...,0.566641,False,123.971421
7,"When making a U-Turn, you should",Check for traffic behind you and indicate your...,The provided text does not specifically discus...,0.663100,True,139.088943
8,Signs that are triangular in shape and with a ...,Caution or warning signs,"Warning signs, which are typically triangular ...",0.731701,True,71.944613
9,"Signs that are round, inverted triangle or oct...",Regulatory signs,Warning signs are typically characterized by a...,0.747711,True,82.784009


# 11. Relevancy Evaluation

In [ ]:
from llama_index.core.evaluation import RelevancyEvaluator

ollama_llm = Ollama(model="llama3.3", request_timeout=300)
evaluator = RelevancyEvaluator(llm=ollama_llm)

eval_results = []

for i in tqdm(range(len(df))):
    eval_result = await evaluator.aevaluate(
        query=df.loc[i, "Question"],
        response=df.loc[i, "AI"],
        contexts=[df.loc[i, "Context"]]  
    )
    eval_results.append(eval_result.passing)

df['Eval'] = eval_results

total_items = len(df)
passing_items = df['Eval'].sum()
score = f"Score: {passing_items}/{total_items}"
percentage = passing_items / total_items if total_items > 0 else 0
print(score)
print(f"Percentage: {percentage:.2%}")
display(df[['Question', 'Answer', 'AI', 'Eval', 'Inference_Time']])

100%|██████████| 60/60 [1:50:05<00:00, 110.10s/it]


Score: 46/60
Percentage: 76.67%


,Question,Answer,AI,Eval,Inference_Time
0,Traffic Jam can be prevented if you,Keep opposing lanes open,"To prevent traffic jams, it's essential to pra...",True,167.875098
1,When making a right turn you should,Stay on the outermost lane of the road then si...,"When making a right turn, you should drive in ...",False,155.435959
2,"When you intend to turn right or left, signal ...",25 meters before you intend to make your turn,"30 meters before turning. \n\nHowever, since t...",True,107.802828
3,"At an intersection with a traffic light, make ...",The green light is on and there is a left turn...,"At an intersection with a traffic light, make ...",True,112.805041
4,Graft and corruption in the traffic enforcemen...,Self disciplined by drivers and obeying traffi...,To eliminate graft and corruption in the traff...,True,171.503720
5,"On a four(4) lane road with single white line,...",Overtake by passing over the solid white line,"On a four-lane road with a single white line, ...",False,120.294474
6,A double solid yellow line with broken white l...,Absolutely no overtaking,A double solid yellow line with a broken white...,False,123.971421
7,"When making a U-Turn, you should",Check for traffic behind you and indicate your...,The provided text does not specifically discus...,True,139.088943
8,Signs that are triangular in shape and with a ...,Caution or warning signs,"Warning signs, which are typically triangular ...",True,71.944613
9,"Signs that are round, inverted triangle or oct...",Regulatory signs,Warning signs are typically characterized by a...,True,82.784009
